<a href="https://colab.research.google.com/github/joselvira/BiomecanicaPython/blob/master/Notebooks/Transformar_Bases_de_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CÓMO TRANSFORMAR LA ORIENTACIÓN O ESTRUCTURA DE LAS BASES DE DATOS

Jose Luis López Elvira ([enlace a más recursos en GitHub](https://github.com/joselvira/BiomecanicaPython))

Última modificación 17/07/2025

Normalmente utilizamos bases de datos en las que vamos introduciendo resultados con una estructura de sujetos en filas y variables en columnas. Esta disposición podemos denominarla **horizontal** (en inglés se suele llamar *wide* o *tidy*) y es muy cómoda para trabajar con datos sencillos. Se dice que el formato tidy debe cumplir tres requisitos:


1.   Cada variable en una columna.
2.   Cada observación en una fila.
3.   Cada celda contiene una única medida.

Sin embargo, cuando empezamos a introducir distintos factores (como pre y postest, o distintos grupos de intervención) esta orientación se queda corta. Programas estadísticos como el SPSS y algunas librerías de python requieren que los factores se apilen en formato **vertical** (en inglés *long*) cuando queremos hacer comparaciones con ANOVAS por ejemplo. Si tenemos pocos factores y pocos datos se puede hacer a mano directamente en la hoja de cálculo, pero si tenemos muchos y variados factores resulta más complicado y tedioso. Además, este formato es más práctico a la hora de hacer representaciones gráficas rápidas teniendo en cuenta los factores.
Puedes encontrar más información sobre estos formatos en la [guía de visualización de datos de la Unión Europea](https://data.europa.eu/apps/data-visualisation-guide/intro-to-tidy-data)

También hay otro formato **mixto**, que coloca las etiquetas de clasificación de los factores en columnas, y mantiene cada variable en una columna diferente.

En este apartado veremos cómo podemos hacer la adaptación de bases de datos con orientación horizontal a vertical y al revés. Seguramente no se contemplarán todas las opciones posibles que se puedan dar en todos los estudios, pero al menos se abarcan las que pueden ser más frecuentes, y pueden servir como base por donde empezar para adaptarlas a las necesidades propias.

## DE HORIZONTAL A VERTICAL (de *wide* a *long*)

### Ejemplo sencillo

Creamos una base de datos sencilla con orientación horizontal.

Tiene un factor tiempo con tres niveles y un factor grupo con dos niveles.

In [1]:
import numpy as np
import pandas as pd

n = 3
Tiempo = ['base', 'pre', 'post']
Grupo = ['GE', 'GC']

np.random.seed(456) #fija la aleatoriedad

#Creamos datos aleatorios para cada grupo de datos
GEBase = np.random.normal(7.3, size=n)
GEPre = np.random.normal(7.4, size=n)
GEPost = np.random.normal(10.1, size=n)
GCBase = np.random.normal(7.5, size=n)
GCPre = np.random.normal(7.5, size=n)
GCPost = np.random.normal(7.5, size=n)


#Los junta en un dataframe
df = pd.DataFrame({'SujID': np.arange(n*len(Grupo))+1,
                   'Grupo': np.repeat(Grupo, n),
                   'Medida_base': np.hstack([GEBase, GCBase]),
                   'Medida_pre': np.hstack([GEPre, GCPre]),
                   'Medida_post': np.hstack([GEPost, GCPost]),
                   })

df

,SujID,Grupo,Medida_base,Medida_pre,Medida_post
0,1,GE,6.631871,7.968692,10.401966
1,2,GE,6.801790,8.750509,10.549483
2,3,GE,7.918576,9.029589,9.754189
3,4,GC,7.184769,6.388154,9.444702
4,5,GC,5.484029,7.737851,9.415676
5,6,GC,6.369769,7.174870,8.420348


Ahora lo colocamos en disposición vertical.

In [2]:
#primero fuerza a poner los no del sujeto con 2 caracteres, si no los desordena
df['SujID'] = ['{0:02d}'.format(int(x)) for x in df['SujID']]
nomVars='Medida'
dfFactores = pd.melt(df, id_vars=['SujID', 'Grupo'], value_vars=df.filter(regex=nomVars).columns, var_name='NomVariable', value_name=nomVars)

dfFactores

,SujID,Grupo,NomVariable,Medida
0,01,GE,Medida_base,6.631871
1,02,GE,Medida_base,6.801790
2,03,GE,Medida_base,7.918576
3,04,GC,Medida_base,7.184769
4,05,GC,Medida_base,5.484029
5,06,GC,Medida_base,6.369769
6,01,GE,Medida_pre,7.968692
7,02,GE,Medida_pre,8.750509
8,03,GE,Medida_pre,9.029589
9,04,GC,Medida_pre,6.388154


Ahora se expande la variable creada NomVariable para que guarde en otra columna el factor tiempo.


In [3]:
dfFactores['Tiempo'] = dfFactores['NomVariable'].str.split('_', expand=True)[1]

Por último, se pueden ordenar las columnas.

In [4]:
dfFactores = dfFactores.reindex (columns=['SujID', 'NomVariable', 'Grupo', 'Tiempo', 'Medida'])

dfFactores

,SujID,NomVariable,Grupo,Tiempo,Medida
0,01,Medida_base,GE,base,6.631871
1,02,Medida_base,GE,base,6.801790
2,03,Medida_base,GE,base,7.918576
3,04,Medida_base,GC,base,7.184769
4,05,Medida_base,GC,base,5.484029
5,06,Medida_base,GC,base,6.369769
6,01,Medida_pre,GE,pre,7.968692
7,02,Medida_pre,GE,pre,8.750509
8,03,Medida_pre,GE,pre,9.029589
9,04,Medida_pre,GC,pre,6.388154


### Otro ejemplo con más variables dependientes

In [6]:
import numpy as np
import pandas as pd

n = 3
Tiempo = ['pre', 'post']
Grupo = ['GE', 'GC']

np.random.seed(456) #fija la aleatoriedad

#Creamos datos aleatorios para cada grupo de datos
GEPre1 = np.random.normal(7.4, size=n)
GEPost1 = np.random.normal(10.1, size=n)
GCPre1 = np.random.normal(7.5, size=n)
GCPost1 = np.random.normal(7.5, size=n)

GEPre2 = np.random.normal(27.4, size=n)
GEPost2 = np.random.normal(30.1, size=n)
GCPre2 = np.random.normal(27.5, size=n)
GCPost2 = np.random.normal(27.5, size=n)


#Los junta en un dataframe
df2 = pd.DataFrame({'SujID': np.arange(n*len(Grupo))+1,
                   'Grupo': np.repeat(Grupo, n),
                   'Medida1_pre': np.hstack([GEPre1, GCPre1]),
                   'Medida1_post': np.hstack([GEPost1, GCPost1]),
                   'Medida2_pre': np.hstack([GEPre2, GCPre2]),
                   'Medida2_post': np.hstack([GEPost2, GCPost2]),
                   })

df2

,SujID,Grupo,Medida1_pre,Medida1_post,Medida2_pre,Medida2_post
0,1,GE,6.731871,10.668692,26.288154,32.044702
1,2,GE,6.901790,11.450509,27.637851,32.015676
2,3,GE,8.018576,11.729589,27.074870,31.020348
3,4,GC,7.801966,7.184769,28.436398,27.411248
4,5,GC,7.949483,5.484029,28.657552,26.528515
5,6,GC,7.154189,6.369769,27.393792,27.115940


Ahora se pasa a formato vertical por factores. Se puede poner separando cada variable dependiente en una columna, o todo en vertical incluidas las variables dependientes (facilita crear gráficos por agrupaciones).

#### 1- Separando cada variable dependiente en una columna

In [7]:
#primero fuerza a poner los no del sujeto con 2 caracteres, si no los desordena
df2['SujID'] = ['{0:02d}'.format(int(x)) for x in df2['SujID']]
nomVars=['Medida1', 'Medida2']

dfFactores2 = pd.DataFrame()
for var in nomVars:
  provis = pd.melt(df2, id_vars=['SujID', 'Grupo'], value_vars=df2.filter(regex=var).columns, var_name='Index', value_name=var)
  dfFactores2[var] = provis.iloc[:, -1]

#Ahora hay que añadirle las otras variables
dfFactores2['SujID'] = provis['SujID']
#dfFactores2['NomVariable'] = provis['Index']
dfFactores2['Tiempo'] = provis['Index'].str.split('_', expand=True)[1]
dfFactores2['Grupo'] = provis['Grupo']

#Y se reordenan las columnas
dfFactores2 = dfFactores2.reindex (columns=['SujID', 'Grupo', 'Tiempo'] + nomVars)
dfFactores2
#provis

,SujID,Grupo,Tiempo,Medida1,Medida2
0,01,GE,pre,6.731871,26.288154
1,02,GE,pre,6.901790,27.637851
2,03,GE,pre,8.018576,27.074870
3,04,GC,pre,7.801966,28.436398
4,05,GC,pre,7.949483,28.657552
5,06,GC,pre,7.154189,27.393792
6,01,GE,post,10.668692,32.044702
7,02,GE,post,11.450509,32.015676
8,03,GE,post,11.729589,31.020348
9,04,GC,post,7.184769,27.411248


#### 2- Todo en vertical incluidas las variables dependientes

En value_vars introducir las variables que haya que ordenar, se puede hacer con las distintas opciones de Pandas.

In [8]:

dfFactores2_2 = pd.melt(df2, id_vars=['SujID', 'Grupo'], value_vars=df2.filter(regex='Medida').columns, var_name='Index', value_name='Medida')

#A partir de la variable Index, crea la columna Tiempo y NomVariable
dfFactores2_2['Tiempo'] = dfFactores2_2['Index'].str.split('_', expand=True)[1]
dfFactores2_2['NomVariable'] = dfFactores2_2['Index'].str.split('_', expand=True)[0]

#Y se reordenan las columnas. De paso se elimina la variable Index
dfFactores2_2 = dfFactores2_2.reindex (columns=['SujID', 'Grupo', 'Tiempo', 'NomVariable', 'Medida'])

dfFactores2_2

,SujID,Grupo,Tiempo,NomVariable,Medida
0,01,GE,pre,Medida1,6.731871
1,02,GE,pre,Medida1,6.901790
2,03,GE,pre,Medida1,8.018576
3,04,GC,pre,Medida1,7.801966
4,05,GC,pre,Medida1,7.949483
5,06,GC,pre,Medida1,7.154189
6,01,GE,post,Medida1,10.668692
7,02,GE,post,Medida1,11.450509
8,03,GE,post,Medida1,11.729589
9,04,GC,post,Medida1,7.184769


### Ejemplo más complejo con más variables

Ahora cargamos una base de datos de ejemplo, que contiene una muestra que realiza un experimento en el que se realiza un test de precisión en una diana, otro de control postural y otro test de salto vertical.

In [9]:
import pandas as pd

#Cargar la base de datos de ejemplo
urlDatos='https://raw.githubusercontent.com/joselvira/BiomecanicaPython/master/Datos/BasesDatos/BaseDatosDianas_Fake.xlsx'
nombreHoja = 'DatosHorizontal'

dfDatos = pd.read_excel(urlDatos, sheet_name=nombreHoja, index_col=None)
dfDatos

,SujID,Grupo,Edad,Talla,Masa,Puntos_pre,Equilibrio_pre,Salto_pre,Puntos_post,Equilibrio_post,Salto_post,Puntos_retest,Equilibrio_retest,Salto_retest
0,1,Entrenado,13.508680,1.609190,53.265265,24,11.971867,21.572761,33,40.906041,23.049010,33,14.211841,21.909154
1,2,Entrenado,14.108241,1.614911,55.770581,31,11.103282,21.831651,37,7.443853,23.249798,31,14.580621,22.126124
2,3,Entrenado,15.127287,1.793211,59.538736,27,11.377356,21.788410,33,7.930517,23.281648,24,13.763388,22.071827
3,4,Entrenado,14.284315,1.804726,51.641111,22,12.275727,22.458442,25,8.371687,23.092659,31,14.672547,22.151674
4,5,Entrenado,13.285982,1.694118,56.992277,25,11.441037,22.107812,28,8.138590,22.927019,25,13.504805,21.755714
5,6,Entrenado,14.416425,1.672931,51.258039,23,11.990435,22.235000,40,8.587306,23.105766,35,14.126287,22.110633
6,7,Entrenado,13.805980,1.781063,60.598647,22,12.004513,21.824378,30,7.813307,22.921988,32,13.769698,21.769799
7,8,Entrenado,14.066872,1.843984,53.252213,24,12.128285,21.802244,30,7.711281,22.858167,28,13.896297,22.300287
8,9,Entrenado,14.157171,1.696319,52.817500,24,11.881908,21.872469,31,7.834597,22.832338,32,13.239037,22.132844
9,10,Entrenado,14.207242,1.654424,55.616288,21,11.289391,21.969301,29,8.816140,22.646438,31,13.257044,21.978297


Hacemos una primera transformación.

In [10]:
#primero fuerza a poner los no del sujeto con 2 caracteres, si no los desordena
dfDatos['SujID'] = ['{0:02d}'.format(int(x)) for x in dfDatos['SujID']]
varDescrip = ['SujID',	'Grupo',	'Edad',	'Talla', 'Masa']
var=['Puntos', 'Equilibrio', 'Salto']
dfFactores = pd.melt(dfDatos, id_vars=varDescrip, value_vars=dfDatos.columns[5:], var_name='Index', value_name='Medida')

#A partir de la variable Index, crea la columna Tiempo y NomVariable
dfFactores['Tiempo'] = dfFactores['Index'].str.split('_', expand=True)[1]
dfFactores['NomVariable'] = dfFactores['Index'].str.split('_', expand=True)[0]

#Y se reordenan las columnas. De paso se elimina la variable Index
dfFactores = dfFactores.reindex (columns=varDescrip +['Tiempo', 'NomVariable', 'Medida'])

dfFactores

,SujID,Grupo,Edad,Talla,Masa,Tiempo,NomVariable,Medida
0,01,Entrenado,13.508680,1.609190,53.265265,pre,Puntos,24.000000
1,02,Entrenado,14.108241,1.614911,55.770581,pre,Puntos,31.000000
2,03,Entrenado,15.127287,1.793211,59.538736,pre,Puntos,27.000000
3,04,Entrenado,14.284315,1.804726,51.641111,pre,Puntos,22.000000
4,05,Entrenado,13.285982,1.694118,56.992277,pre,Puntos,25.000000
...,...,...,...,...,...,...,...,...
265,26,Control,14.470242,1.683536,70.255573,retest,Salto,17.945808
266,27,Control,14.326451,1.673556,63.492148,retest,Salto,18.034146
267,28,Control,14.316783,1.784191,65.779244,retest,Salto,17.659063
268,29,Control,14.880468,1.513507,61.497886,retest,Salto,17.996960


## DE VERTICAL A HORIZONTAL de long a wide)

### Pasa de disposición vertical a horizontal mixta

En este caso mantenemos en horizontal el factor tiempo en columnas y mantiene en vertical el factor grupo, como en los análisis del SPSS.

Empezamos con un ejemplo sencillo de base de datos en disposición vertical para pasarlo a horizontal.

Tiene un factor tiempo con tres niveles y un factor grupo con dos niveles. Los datos originales se ordenan alternando el grupo (GE, GE, GC, GC, GE, GE, GC,GC...) mientras que el factor tiempo se coloca seguido (pre, pre, pre, pre, pre, pre, post, post, post, post, post, ...)

In [11]:
import numpy as np
import pandas as pd

n = 3
Tiempo = ['base', 'pre', 'post']
Grupo = ['GE', 'GC']

np.random.seed(456) #fija la aleatoriedad

#Creamos datos aleatorios para cada grupo de datos
GEBase = np.random.normal(7.3, size=n)
GEPre = np.random.normal(7.4, size=n)
GEPost = np.random.normal(10.1, size=n)
GCBase = np.random.normal(7.5, size=n)
GCPre = np.random.normal(7.5, size=n)
GCPost = np.random.normal(7.5, size=n)

#Los junta en un dataframe
df = pd.DataFrame({'SujID': np.tile(np.arange(n*len(Grupo))+1, len(Tiempo)),
                   'Grupo': np.tile(np.repeat(Grupo, len(Tiempo)), n),
                   'Tiempo': np.repeat(Tiempo, n*len(Grupo)),
                   'Medida': np.r_[GEBase, GCBase, GEPre, GCPre, GEPost, GCPost]
                   })

df


,SujID,Grupo,Tiempo,Medida
0,1,GE,base,6.631871
1,2,GE,base,6.801790
2,3,GE,base,7.918576
3,4,GC,base,7.184769
4,5,GC,base,5.484029
5,6,GC,base,6.369769
6,1,GE,pre,7.968692
7,2,GE,pre,8.750509
8,3,GE,pre,9.029589
9,4,GC,pre,6.388154


Ahora pasamos el factor tiempo a una disposición horizontal, manteniendo el factor grupo en vertical, como se hace en SPSS.

In [12]:
#primero fuerza a poner los no del sujeto con 2 caracteres, si no los desordena
df['SujID'] = ['{0:02d}'.format(int(x)) for x in df['SujID']]
#expande las variables dependientes a partir del nº de sujeto
dfHorizMixto = df.pivot(index='SujID', columns='Tiempo', values=['Medida'])
#ordena las variables como debe
dfHorizMixto = dfHorizMixto.reindex(columns=['base', 'pre', 'post'], level='Tiempo')
#aplasta el encabezado en una línea
dfHorizMixto.columns = dfHorizMixto.columns.map('_'.join).str.strip()
nonVarNuevas = list(dfHorizMixto.columns)
#pasa los sujetos de índice a columna
dfHorizMixto.reset_index(level=0, inplace=True)

varDescriptivas = ['Grupo']
for var in varDescriptivas:
    dfHorizMixto[var] = df.loc[:dfHorizMixto.shape[0]-1, var]
dfHorizMixto = dfHorizMixto.reindex (columns= ['SujID'] + varDescriptivas + nonVarNuevas)
dfHorizMixto

,SujID,Grupo,Medida_base,Medida_pre,Medida_post
0,01,GE,6.631871,7.968692,10.401966
1,02,GE,6.801790,8.750509,10.549483
2,03,GE,7.918576,9.029589,9.754189
3,04,GC,7.184769,6.388154,9.444702
4,05,GC,5.484029,7.737851,9.415676
5,06,GC,6.369769,7.174870,8.420348


### Ejemplo con más variables

Ahora cargamos una base de datos de ejemplo, que contiene una muestra que realiza un experimento en el que se realiza un test de precisión en una diana, otro de control postural y otro test de salto vertical.

In [13]:
import pandas as pd

#Cargar la base de datos de ejemplo
urlDatos='https://raw.githubusercontent.com/joselvira/BiomecanicaPython/master/Datos/BasesDatos/BaseDatosDianas_Fake.xlsx'
nombreHoja = 'DatosVertical'

dfDatos = pd.read_excel(urlDatos, sheet_name=nombreHoja, index_col=None)
dfDatos

,SujID,SujGrupo,Grupo,Tiempo,Sexo,Edad,Talla,Masa,Puntos,Equilibrio,Salto
0,1,1,Entrenado,pre,Mixto,13.508680,1.609190,53.265265,24,11.971867,21.572761
1,2,2,Entrenado,pre,Mixto,14.108241,1.614911,55.770581,31,11.103282,21.831651
2,3,3,Entrenado,pre,Mixto,15.127287,1.793211,59.538736,27,11.377356,21.788410
3,4,4,Entrenado,pre,Mixto,14.284315,1.804726,51.641111,22,12.275727,22.458442
4,5,5,Entrenado,pre,Mixto,13.285982,1.694118,56.992277,25,11.441037,22.107812
...,...,...,...,...,...,...,...,...,...,...,...
85,26,11,Control,retest,Mixto,14.470242,1.683536,70.255573,28,12.225928,17.945808
86,27,12,Control,retest,Mixto,14.326451,1.673556,63.492148,29,13.142626,18.034146
87,28,13,Control,retest,Mixto,14.316783,1.784191,65.779244,30,11.490003,17.659063
88,29,14,Control,retest,Mixto,14.880468,1.513507,61.497886,25,11.388211,17.996960


In [14]:
#primero fuerza a poner los no del sujeto con 2 caracteres, si no los desordena
dfDatos['SujID'] = ['{0:02d}'.format(int(x)) for x in dfDatos['SujID']]
#expande las variables dependientes a partir del nº de sujeto
dfHorizMixto = dfDatos.pivot(index='SujID', columns='Tiempo', values=['Puntos', 'Equilibrio', 'Salto'])
#ordena las variables como debe
dfHorizMixto = dfHorizMixto.reindex(columns=['pre', 'post', 'retest'], level='Tiempo')
#aplasta el encabezado en una línea
dfHorizMixto.columns = dfHorizMixto.columns.map('_'.join).str.strip()
nonVarNuevas = list(dfHorizMixto.columns)
#pasa los sujetos de índice a columna
dfHorizMixto.reset_index(level=0, inplace=True)

varDescriptivas = ['SujGrupo', 'Grupo', 'Tiempo', 'Sexo', 'Edad', 'Talla', 'Masa']
for var in varDescriptivas:
    dfHorizMixto[var] = dfDatos.loc[:dfHorizMixto.shape[0]-1, var]
dfHorizMixto = dfHorizMixto.reindex (columns= ['SujID'] + varDescriptivas + nonVarNuevas)

dfHorizMixto

,SujID,SujGrupo,Grupo,Tiempo,Sexo,Edad,Talla,Masa,Puntos_pre,Puntos_post,Puntos_retest,Equilibrio_pre,Equilibrio_post,Equilibrio_retest,Salto_pre,Salto_post,Salto_retest
0,01,1,Entrenado,pre,Mixto,13.508680,1.609190,53.265265,24.0,33.0,33.0,11.971867,18.906041,14.211841,21.572761,23.049010,21.909154
1,02,2,Entrenado,pre,Mixto,14.108241,1.614911,55.770581,31.0,37.0,31.0,11.103282,7.443853,14.580621,21.831651,23.249798,22.126124
2,03,3,Entrenado,pre,Mixto,15.127287,1.793211,59.538736,27.0,33.0,24.0,11.377356,7.930517,13.763388,21.788410,23.281648,22.071827
3,04,4,Entrenado,pre,Mixto,14.284315,1.804726,51.641111,22.0,25.0,31.0,12.275727,8.371687,14.672547,22.458442,23.092659,22.151674
4,05,5,Entrenado,pre,Mixto,13.285982,1.694118,56.992277,25.0,28.0,25.0,11.441037,8.138590,13.504805,22.107812,22.927019,21.755714
5,06,6,Entrenado,pre,Mixto,14.416425,1.672931,51.258039,23.0,40.0,35.0,11.990435,8.587306,14.126287,22.235000,23.105766,22.110633
6,07,7,Entrenado,pre,Mixto,13.805980,1.781063,60.598647,22.0,30.0,32.0,12.004513,7.813307,13.769698,21.824378,22.921988,21.769799
7,08,8,Entrenado,pre,Mixto,14.066872,1.843984,53.252213,24.0,30.0,28.0,12.128285,7.711281,13.896297,21.802244,22.858167,22.300287
8,09,9,Entrenado,pre,Mixto,14.157171,1.696319,52.817500,24.0,31.0,32.0,11.881908,7.834597,13.239037,21.872469,22.832338,22.132844
9,10,10,Entrenado,pre,Mixto,14.207242,1.654424,55.616288,21.0,29.0,31.0,11.289391,8.816140,13.257044,21.969301,22.646438,21.978297
